In [1]:
import os
import sys

import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sys.path.insert(0, '../src')

from utils import *

In [3]:
sns.set(rc={'figure.figsize':(20,5)})

In [4]:
mbit_rate = 1/125000

two_four_fp = '../data/240p/'
three_six_fp = '../data/360p/'
four_eight_fp = '../data/480p/'
seven_two_fp = '../data/720p/'
ten_eight_fp = '../data/1080p/'

two_four_dir = os.listdir(two_four_fp)
three_six_dir = os.listdir(three_six_fp)
four_eight_dir = os.listdir(four_eight_fp)
seven_two_dir = os.listdir(seven_two_fp)
ten_eight_dir = os.listdir(ten_eight_fp)

df_240_lst = [add_resolution(two_four_fp + fp, '240p') for fp in two_four_dir]
df_360_lst = [add_resolution(three_six_fp + fp, '360p') for fp in three_six_dir]
df_480_lst = [add_resolution(four_eight_fp + fp, '480p') for fp in four_eight_dir]
df_720_lst = [add_resolution(seven_two_fp + fp, '720p') for fp in seven_two_dir]
df_1080_lst = [add_resolution(ten_eight_fp + fp, '1080p') for fp in ten_eight_dir]

chunk_240_lst = sum([chunk_data(df) for df in df_240_lst], [])
chunk_360_lst = sum([chunk_data(df) for df in df_360_lst], [])
chunk_480_lst = sum([chunk_data(df) for df in df_480_lst], [])
chunk_720_lst = sum([chunk_data(df) for df in df_720_lst], [])
chunk_1080_lst = sum([chunk_data(df) for df in df_1080_lst], [])

## Aggregate Features
In our EDA, we saw significant differences in aggregate statistics such as the mean and standard deviation. We reconfirm this by taking our chunked data and performing said operations. There seems to be alot of potential colinearity between the bytes and packet stream statistics (strong positive correlation). In our model, we chose to take the aggregate features of just the download stream of bytes. 

In [57]:
def agg_features_all(df_lst):
  download_byte_feat = [np.array([np.mean(df['2->1Bytes']), np.std(df['2->1Bytes'])]) * mbit_rate for df in df_lst]
  download_pkt_feat = [np.array([np.mean(df['2->1Pkts']), np.std(df['2->1Pkts'])]) for df in df_lst]
  upload_byte_feat = [np.array([np.mean(df['1->2Bytes']), np.std(df['1->2Bytes'])]) * mbit_rate for df in df_lst]
  upload_pkt_feat = [np.array([np.mean(df['1->2Pkts']), np.std(df['1->2Pkts'])]) for df in df_lst]
  return [download_byte_feat, download_pkt_feat, upload_byte_feat, upload_pkt_feat]

In [58]:
np.mean(agg_features_all(chunk_240_lst), axis=1)

array([[2.68929318e-01, 1.15340577e+00],
       [2.68640812e+01, 1.09422755e+02],
       [2.01655931e-02, 6.47301899e-02],
       [1.77151682e+01, 6.06875203e+01]])

In [59]:
np.mean(agg_features_all(chunk_360_lst), axis=1)

array([[3.47693444e-01, 1.62085922e+00],
       [3.32106015e+01, 1.50414272e+02],
       [2.10986046e-02, 8.15594338e-02],
       [1.89911651e+01, 7.70355700e+01]])

In [61]:
np.mean(agg_features_all(chunk_480_lst), axis=1)

array([[6.47737180e-01, 2.45569632e+00],
       [6.28800533e+01, 2.31008800e+02],
       [4.12340273e-02, 1.27121836e-01],
       [3.73126416e+01, 1.23173373e+02]])

In [62]:
np.mean(agg_features_all(chunk_720_lst), axis=1)

array([[1.32797336e+00, 3.51618586e+00],
       [1.24230574e+02, 3.25938092e+02],
       [6.96803972e-02, 1.69834253e-01],
       [6.61051448e+01, 1.65536005e+02]])

In [63]:
np.mean(agg_features_all(chunk_1080_lst), axis=1)

array([[1.95912153e+00, 4.25834585e+00],
       [1.89135641e+02, 4.00306507e+02],
       [1.06824435e-01, 2.15001592e-01],
       [1.05286826e+02, 2.08072070e+02]])

## Peak Related Aggregate Features
Peaks were a strong point of focus in our EDA as there might be strong potential features to be extracted. We start by taking the definition of a "peak" from before: 1 standard deviation above the mean. The 1 factor is a heuristic we landed upon after experimentation. Knowing how many spikes is very useful but also knowing the time delay between each spikes is useful. We will also apply a signal processing technique of evenly spacing out spikes and then taking the time delay between them. We also experiement with a hard threshold of 5 Mbps as a peak. We realize that this will not be applicable to a wide variety of other users as people's internet speed can wildly vary.

In [202]:
def get_peak_loc(df, col, invert=False):
  'invert arg allows you to get values not considered peaks'
  df_avg = df[col].mean()
  df_std = df[col].std()
  
  threshold = df_avg + (1 * df_std)
  if invert:
    return np.array(df[col] < threshold)
  
  else:
    return np.array(df[col] > threshold)
  
def hard_threshold_peaks(df, col, thresh):
    x = df[col]
    peaks, _ = sp.signal.find_peaks(x, height=thresh)
    return peaks
  
def peak_time_diff(df, col):
  '''
  mess around with the different inputs for function. 
  variance seems to inflate the difference betweent the two the most with litte
  to no data manipulation. however, currently trying things like
  squaring the data before taking the aggregate function to exaggerate
  differences (moderate success??)
  '''
  #peaks = df[get_peak_loc(df, col)]
  peaks = df.iloc[hard_threshold_peaks(df, col, 625000)]
  peaks['Time'] = peaks['Time'] - peaks['Time'].min()
  time_diff = np.diff(peaks['Time'])
  if len(peaks) <= 0:
    return [-1, 0, -1]
  
  return [np.mean(peaks)['2->1Bytes'] * mbit_rate, len(peaks), 120 / len(peaks)]

In [178]:
%%time
peaks_240 = [peak_time_diff(df, '2->1Bytes') for df in chunk_240_lst]
peaks_360 = [peak_time_diff(df, '2->1Bytes') for df in chunk_360_lst]
peaks_480 = [peak_time_diff(df, '2->1Bytes') for df in chunk_480_lst]
peaks_720 = [peak_time_diff(df, '2->1Bytes') for df in chunk_720_lst]
peaks_1080 = [peak_time_diff(df, '2->1Bytes') for df in chunk_1080_lst]

Wall time: 22.8 s


In [179]:
np.mean(peaks_240, axis=0)

array([ 6.8263235 ,  3.09638554, 48.36890419])

In [180]:
np.mean(peaks_360, axis=0)

array([10.3893675 ,  3.46987952, 40.3350545 ])

In [181]:
np.mean(peaks_480, axis=0)

array([12.35970354,  4.68674699, 32.56751682])

In [182]:
np.mean(np.nan_to_num(peaks_720), axis=0)

array([13.09373498,  9.32926829, 19.91668919])

In [183]:
np.mean(peaks_1080, axis=0)

array([13.26072742, 15.24096386, 11.27606313])

## Spectral Features

In [295]:
def spectral_features(df, col):
    """
    welch implemention of spectral features
    resample the data before inputting (might change prereq depending on
    resource allocation)
    """

    f, Pxx_den = sp.signal.welch(df[col], fs=2)
    Pxx_den = np.sqrt(Pxx_den)

    peaks = sp.signal.find_peaks(Pxx_den)[0]
    prominences = sp.signal.peak_prominences(Pxx_den, peaks)[0]

    idx_max = prominences.argmax()
    loc_max = peaks[idx_max]

    return [f[loc_max], Pxx_den[loc_max], prominences[idx_max]]
  
def power_density(df, bins):
  
  f_temp, Pxx_temp = sp.signal.welch(df['pkt_size'], fs=.5) 
  Pxx_temp = np.sqrt(Pxx_temp)
  freq = np.linspace(0, np.max(f_temp) + .01, num=bins) - .001
  total = np.trapz(y=Pxx_temp, x=f_temp)
  temp_lst = []
  
  for i in np.arange(len(freq) - 1):

    f_lower = np.where(f_temp >= freq[i])
    f_upper = np.where(f_temp < freq[i+1] )
    selected_range = np.intersect1d(f_lower, f_upper)

    pxx_den = np.trapz(y=Pxx_temp[selected_range], x=f_temp[selected_range]) / total
    temp_lst.append(pxx_den)

  return temp_lst

def temp_name(df):
  f_temp, Pxx_temp = sp.signal.welch(df['pkt_size'], fs=2) 
  return np.sum((Pxx_temp/np.sum(Pxx_temp)) * f_temp)

In [287]:
%%time
size='500ms'
resample_240 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_240_lst]
resample_360 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_360_lst]
resample_480 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_480_lst]
resample_720 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_720_lst]
resample_1080 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_1080_lst]

Wall time: 31.1 s


In [302]:
%%time
size='2000ms'

resample_240 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_240_lst]
resample_360 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_360_lst]
resample_480 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_480_lst]
resample_720 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_720_lst]
resample_1080 = [convert_ms_df(df).resample(size, on='Time').sum() for df in chunk_1080_lst]

Wall time: 41 s


In [303]:
pxx_den_240 = [power_density(df, 4) for df in resample_240]
pxx_den_360 = [power_density(df, 4) for df in resample_360]
pxx_den_480 = [power_density(df, 4) for df in resample_480]
pxx_den_720 = [power_density(df, 4) for df in resample_720]
pxx_den_1080 = [power_density(df, 4) for df in resample_1080]

In [336]:
pxx_den_df = pd.DataFrame({
  "240p": np.mean(pxx_den_240, axis=0),
  "360p": np.mean(pxx_den_360, axis=0),
  "480p": np.mean(pxx_den_480, axis=0),
  "720p": np.mean(pxx_den_720, axis=0),
  "1080p": np.mean(pxx_den_1080, axis=0)
}).T

pxx_den_df.columns = ['[-.001, .086)', '[.086, .172)', '[.172, .259)']
pxx_den_df['St_dev'] = [
  np.std(np.mean(pxx_den_240, axis=0)),
  np.std(np.mean(pxx_den_360, axis=0)),
  np.std(np.mean(pxx_den_480, axis=0)),
  np.std(np.mean(pxx_den_720, axis=0)), 
  np.std(np.mean(pxx_den_1080, axis=0))]

pxx_den_df


,"[-.001, .086)","[.086, .172)","[.172, .259)",St_dev
240p,0.309454,0.325631,0.297976,0.011345
360p,0.318108,0.316864,0.296115,0.010087
480p,0.316858,0.328683,0.284150,0.018835
720p,0.257639,0.342087,0.333840,0.038015
1080p,0.223995,0.330782,0.380637,0.065342


In [324]:
np.linspace(0, .25 + .01, num=4) - .001

array([-0.001     ,  0.08566667,  0.17233333,  0.259     ])

In [315]:
(np.mean(pxx_den_360, axis=0))

array([0.31810831, 0.31686423, 0.29611528])

In [316]:
(np.mean(pxx_den_480, axis=0))

array([0.31685758, 0.32868302, 0.28414961])

In [317]:
(np.mean(pxx_den_720, axis=0))

array([0.25763884, 0.34208657, 0.3338403 ])

In [318]:
(np.mean(pxx_den_1080, axis=0))

array([0.22399455, 0.33078182, 0.38063741])

## Chunking & Feature creation

In [ ]:
## wip; need to decide chunk size eventually
## should we also make this chunking feature be our feature creation?

def chunk_data(df, interval=60):

    """
    takes in a filepath to the data you want to chunk and feature engineer
    chunks our data into a specified time interval
    each chunk is then turned into an observation to be fed into our classifier
    """

    df_list = []
    
    df['Time'] = df['Time'] - df['Time'].min()
    
    total_chunks = np.floor(df['Time'].max() / interval).astype(int)

    for chunk in np.arange(total_chunks):
      
        start = chunk * interval
        end = (chunk+1) * interval

        temp_df = (df[(df['Time'] >= start) & (df['Time'] < end)])
        
        df_list.append(temp_df)
        
    return df_list

In [ ]:
def create_features(df, interval=60):

  features = [
    'dwl_peak_freq',
    'dwl_peak_prom',
    'dwl_max_psd',
    'dwl_bytes_avg',
    'dwl_bytes_std',
    'dwl_peak_avg',
    'dwl_peak_var',
    'dwl_peak_std',
    'upl_peak_freq',
    'upl_peak_prom',
    'upl_max_psd',
    'upl_bytes_avg',
    'upl_bytes_std',
    'upl_peak_avg',
    'upl_peak_var',
    'upl_peak_std',
    'IMAN_dwn_time_peak',#'IMAN_up_time_peak',
    'IMAN_dwn_num_peak'#,'IMAN_up_num_peak'
  ]  

  vals = []

  df_chunks = chunk_data(df, interval)

  for chunk in df_chunks:

    preproc = convert_ms_df(chunk, True)
    upl_bytes = preproc[preproc['pkt_src'] == '1'].resample('500ms', on='Time').sum()
    dwl_bytes = preproc[preproc['pkt_src'] == '2'].resample('500ms', on='Time').sum()

    ## spectral features
    dwl_spectral = spectral_features(dwl_bytes, 'pkt_size')
    upl_spectral = spectral_features(upl_bytes, 'pkt_size')
    
    ## aggregate features
    dwl_agg = agg_feat(chunk, '2->1Bytes')
    upl_agg = agg_feat(chunk, '1->2Bytes')
    
    ## peak features
    dwl_peak = peak_time_diff(chunk, '2->1Bytes')
    upl_peak = peak_time_diff(chunk, '1->2Bytes')
    
    ## iman's time between peak 
    iman_dwn_time_peak = np.mean(peak_times(chunk,'2->1Bytes',1000000))
    #iman_up_time_peak = np.mean(peak_times(chunk,'1->2Bytes',50000))

    ## iman's num peak
    iman_dwn_num_peak = num_peaks(chunk,'2->1Bytes',1000000)
    #iman_up_num_peak = num_peaks(chunk,'1->2Bytes',50000)

    feat_val = np.hstack((
      dwl_spectral,
      dwl_agg,
      dwl_peak,
      upl_spectral,
      upl_agg,
      upl_peak,
      iman_dwn_time_peak,
      #iman_up_time_peak,
      iman_dwn_num_peak,
      #iman_up_num_peak
    ))
    
    vals.append(feat_val)
    
  return pd.DataFrame(columns=features, data=vals).fillna(0)

In [ ]:
def create_features_no_split(df, interval=60):

  features = [
    'peak_freq',
    'peak_prom',
    'max_psd',
    'bytes_avg',
    'bytes_std',
    'peak_avg',
    'peak_var',
    'peak_std',
  ]  

  vals = []

  df_chunks = chunk_data(df, interval)

  for chunk in df_chunks:

    preproc = convert_ms_df(chunk, True)

    ## spectral features
    spectral_feat = spectral_features(preproc, 'pkt_size')
    
    ## aggregate features
    aggr_feat = agg_feat(chunk, '2->1Bytes')
    
    ## peak features
    peak_feat = peak_time_diff(chunk, '2->1Bytes')
    
    feat_val = np.hstack((
      spectral_feat,
      aggr_feat,
      peak_feat
    ))
    
    vals.append(feat_val)
    
  return pd.DataFrame(columns=features, data=vals).fillna(0)

In [ ]:
low_feat_no_split = create_features_no_split(stdoan_low, 100)

## Dev Playground

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [ ]:
%%time
low_feat = pd.concat([create_features(df, 100) for df in low_dfs])
med_feat = pd.concat([create_features(df, 100) for df in med_dfs])
high_feat = pd.concat([create_features(df, 100) for df in high_dfs])

In [ ]:
low_feat['resolution'] = np.zeros(len(low_feat))
med_feat['resolution'] = np.zeros(len(med_feat)) + 1
high_feat['resolution'] = np.zeros(len(high_feat)) + 2

In [ ]:
training_split = pd.concat([low_feat, med_feat, high_feat])

In [ ]:
X, y = training_split.drop(columns=['resolution']), training_split['resolution']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 8)

In [ ]:
clf_split = RandomForestClassifier(n_estimators = 5, max_depth = 2, criterion = 'entropy', random_state = 42)
clf_split.fit(X_train, y_train)

In [ ]:
y_pred = clf_split.predict(X_test)

In [ ]:
np.abs(training_split.corr()['resolution']).sort_values(ascending=False)

In [ ]:
(pd.crosstab(y_test, y_pred, rownames=['Actual Resolution'], colnames=['Predicted Resolution']))

In [ ]:
f1_score(y_test, y_pred, average=None)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
features = [
  'dwl_peak_freq',
  'dwl_peak_prom',
  'dwl_max_psd',
  'dwl_bytes_avg',
  'dwl_bytes_std',
  'dwl_peak_avg',
  'dwl_peak_var',
  'dwl_peak_std',
  'upl_peak_freq',
  'upl_peak_prom',
  'upl_max_psd',
  'upl_bytes_avg',
  'upl_bytes_std',
  'upl_peak_avg',
  'upl_peak_var',
  'upl_peak_std',
  'IMAN_dwn_time_peak',#'IMAN_up_time_peak'
            'IMAN_dwn_num_peak']#,'IMAN_up_num_peak']
importances = clf_split.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print(features[i],': ',importances[i])
    

In [ ]:
import dill

In [ ]:
dill.dump(clf_split, open("randomforest_chkpt2.obj", "wb"))

In [ ]:
dill.load(open("randomforest_chkpt2.obj", "rb"))

In [ ]:
pip install scikit-learn

## no split (download focus)

In [ ]:
%%time
low_feat_no_split = create_features_no_split(stdoan_low, 100)
med_feat_no_split = create_features_no_split(stdoan_med, 100)
high_feat_no_split = create_features_no_split(stdoan_high, 100)

In [ ]:
low_feat_no_split['resolution'] = np.zeros(len(low_feat))
med_feat_no_split['resolution'] = np.zeros(len(med_feat)) + 1
high_feat_no_split['resolution'] = np.zeros(len(high_feat)) + 2

In [ ]:
training_no_split = pd.concat([low_feat_no_split, med_feat_no_split, high_feat_no_split])

In [ ]:
X, y = training_no_split.drop(columns=['resolution']), training_no_split['resolution']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 8)

In [ ]:
clf_no_split = RandomForestClassifier(n_estimators = 2, max_depth = 2, criterion = 'entropy', random_state = 42)
clf_no_split.fit(X_train, y_train)

In [ ]:
y_pred = clf_no_split.predict(X_test)

In [ ]:
np.abs(training_no_split.corr()['resolution']).sort_values(ascending=False)

In [ ]:
(pd.crosstab(y_test, y_pred, rownames=['Actual Resolution'], colnames=['Predicted Resolution']))

In [ ]:
f1_score(y_test, y_pred, average=None)